* Hive is not a database. 
* It just poinst to data lying in HDFS.
* Hive is not suitable for OLTP.
* Does not provide row level Insert, Update and delete. 
* Not used where fast response time is required as in RDBMS.

Hive is built on write once and read many concept.

***ARCHITECTURE***
Driver: converts the SQL to MapReduce program.
Compiler: symentatic analysis of the code and aids in coverting SQL to MapReduce.
MetaStore: Stores info on the table
Execution engine: Connected with hadoop system.

### Table creation

* By default Hive creates internal Tables (where the data and metadata is governed by Hive). When dropped both data and metadata is deleted.
* For external tables Hive only governs the metadata


```
CREATE TABLE IF NOT EXISTS table_1 (
    col1 string,
    col2 array<string>,
    col3 string,
    col4 int
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
LINES TERMINATED BY '\n'
STORED AS TEXTFILE; 
```
- default file format is TEXTFILE FORMAT
- default DELIMITED FIELDS TERMINATED BY ' '

-- default location
set hive.metastore.warehouse.dir (user/hive/warehouse)

Change locations while creating table using,
`
LOCATION 'user/aswath/table_1'
`

**SORTING IN HIVE**
1. ORDER BY -- needs to be passed through a single reducer
2. SORT BY -- Sort by orders the data in each reducer (uses two reducers)
3. DISTRIBUTED BY -- Hive uses key value pairs to distributed the data among reducer. It does not perform any sorting. Hence you should follow it by SORT command.
4. CLUSTER BY-- combination of Distribute by + Sort by


## Partitioning

Partitioning is a way of dividing a table into related parts based on the values of particular columns. Reduces the scan time.

### Static partitioning

Add the line in create table statement `PARTITONED BY (departmentname STRING)`

Insert Statment 

`INSERT INTO TABLE department PARTITION (deptname='XYZ')`

**Advantages:** fast

Partitions are created when data is inserted into table. Depending on how you load data you would need partitions. Usually when loading files (big files) into Hive tables static partitions are preferred. That saves your time in loading data compared to dynamic partition. You "statically" add a partition in table and move the file into the partition of the table. Since the files are big they are usually generated in HDFS. You can get the partition column value form the filename, day of date etc without reading the whole big file.


### Dynamic partitioning

Allowing the table create the partitions dynamically based on the values inserted to the column which the user defined as a partition column.

Same create statement. But `set hive.exec.dynamic.partition=true`
and `set hive.exec.dynamic.partition.mode = nonstrict` (disables the restriction on creating a table with atleast one static partition)

`INSERT INTO TABLE  department PARTITION (deptname)`

**Advantage** slow

Incase of dynamic partition whole big file i.e. every row of the data is read and data is partitioned through a MR job into the destination tables depending on certain field in file. So usually dynamic partition are useful when you are doing sort of a ETL flow in your data pipeline. e.g. you load a huge file through a move command into a Table X. then you run a inert query into a Table Y and partition data based on field in table X say day , country. You may want to further run a ETL step to partition the data in country partition in Table Y into a Table Z where data is partitioned based on cities for a particular country only. etc.

Thus depending on your end table or requirements for data and in what form data is produced at source you may choose static or dynamic partition.

## Bucketing

* Another data organizing technique in Hive. Decomposing large dataset into a more manageable one.
* All the same column values of a bucketed column will go into same bucket.
* A partition is a directory and bucket is a actually file with your data.
* The data to a particular bucket is decided by an hashing function.
* Bucketing can sometimes be more efficient when used alone.
* Bucketed map joins are the fastest joins. (Condition: Both joining tables should be bucketed on same column as of joining column and both tables should have equal number of buckets.)


`set hive.enforce.bucketing = true;`

ADD THIS TO THE CREATE STATEMENT

`CLUSTERED BY (location) into 4 buckets`


### Table sampling

Table sampling helps to collect distributed data from different buckets and partition.

```
SELECT
    deptno,
    empname,
    sal,
    location
FROM dept_table
TABLESAMPLE (bucket 2 out of 3 on location)
```

**no_drop** 

`ALTER TABLE emp_tab enable no_drop;`

`ALTER TABLE emp_tab enable PARTITION(dept='HR') no_drop;`

**offline**

no one can query it.

`ALTER TABLE emp_tab enable offline;`


## JOINS

In the last table in query is streamed and the rest are buffered in memory.

### MAP JOINS

The reducer wont be used here. The mapper will be used to join tables. Use smaller tables in map joins.

FULL OUTER MAP JOINS CANNOT BE PERFORMED

### Bucketed map joins

Set the properties.
**Condition:** Both joining tables should be bucketed on same column as of joining column and both tables should have equal number of buckets.


## Views

Virtual table created as a result of a Hive query on a table.

- Views do not contain any data of its own.
- All type of DML operation can be performed on Views.
- Can be created by selecting any number of rows or columns of its base table/taables.
- Once created, the schema of view is frozen and is independent of chnages made to base table schema.
- Vice-versa of above is also True.
- View are read-only
- Drop the table and we can not fire queries on its view.

**Advantages**
- Views can be used to hide underlying table columns from some users.
- Views protect our base table from being accidently dropped or altered.
- Views can help turning the lengthy and complicated query into a one-liner query.


## Indexing

An Index act as reference to records. Used to speed up searching the data.
- Will search for only the portion of data and not whole data set.
- Partition done at HDFS level and indexing is done at table level.

`CREATE INDEX I1 ON TABLE dept(name) AS 'COMPACT' WITH DEFERRED REBUILD;`

COMPACT:

BIT MAP: 

**When to use**
- Dataset is large (GB or more)
- speed is an concern
- Frequent use of where clause in queries

**When not to use**
- Dataset is unique
- No frequent use of where clause in queries.

